In [1]:
import ares
import numpy as np
import matplotlib.pyplot as pl
import distpy

In [2]:
# Independent variables
# redshifts = np.array([0.35, 0.875, 1.125, 1.75, 2.25, 2.75])
redshifts = np.sort(np.array([0.35, 0.875, 1.125, 1.75, 2.25, 2.75, 1.65, 2.5, 3.5, 0.10165, 0.25, 0.45, 0.575, 0.725, 0.9]))


# [0.10165, 0.25, 0.35, 0.45, 0.575, 0.725, 0.9][1.65, 2.5, 3.5]
# MUV = np.arange(-28, -8.8, 0.2)
Ms = np.logspace(7, 12, 50)

# blob 1: the LF. Give it a name, and the function needed to calculate it.
blob_n1 = ['galaxy_smf']
blob_i1 = [('z', redshifts), ('bins', Ms)]
blob_f1 = ['StellarMassFunction']

# # blob 2: the SFE. Same deal.
# blob_n2 = ['fstar']
# blob_i2 = [('z', redshifts), ('Mh', Mh)]
# blob_f2 = ['fstar']
# print(redshifts)

In [3]:
blob_pars = \
{
 'blob_names': [blob_n1],
 'blob_ivars': [blob_i1],
 'blob_funcs': [blob_f1],
 'blob_kwargs': [None],
}

In [14]:
#define the parameters that remain unchanged
base_pars = ares.util.ParameterBundle('emma:model1')
base_pars.update(blob_pars)
base_pars.update({'progress_bar': True, 'debug':True})


# This is important!
# base_pars['pop_calib_lum'] = None

In [15]:
free_pars = \
[
    'pq_func_par0[0]',
    'pq_func_par2[0]', 

    #norm
    'pq_func_par0[1]',
    'pq_func_par2[1]', 

    #gamma
    'pq_func_par0[2]',
    'pq_func_par2[2]', 

    #peak mass
    'pq_func_par0[3]',
    'pq_func_par2[3]', 
]

is_log = [False, False, False, False, False, False, False, False]

from distpy.distribution import UniformDistribution
from distpy.distribution import DistributionSet

ps = DistributionSet()
ps.add_distribution(UniformDistribution(0, 4), 'pq_func_par0[0]')
ps.add_distribution(UniformDistribution(-1, 1),  'pq_func_par2[0]')

ps.add_distribution(UniformDistribution(0.001, 2),   'pq_func_par0[1]')
ps.add_distribution(UniformDistribution(-1, 1),  'pq_func_par2[1]')

ps.add_distribution(UniformDistribution(0, .7),   'pq_func_par0[2]')
ps.add_distribution(UniformDistribution(-3, -0.01),  'pq_func_par2[2]')

ps.add_distribution(UniformDistribution(11.0, 12.6),   'pq_func_par0[3]')
ps.add_distribution(UniformDistribution(0, 1),  'pq_func_par2[3]')

In [16]:
#From Moster2010, table 7
logM_0 = 11.88 #(0.01)
mu = 0.019 #(0.002)
N_0 = 0.0282 #(0.0003)
nu = -0.72 #(0.06)
gamma_0 = 0.556 #0.001
gamma_1 = -0.26 #(0.05)
beta_0 = 1.06 #(0.06)
beta_1 = 0.17 #(0.12)

guesses = \
{
    'pq_func_par0[0]': beta_0,
    'pq_func_par2[0]': beta_1, 

    #norm
    'pq_func_par0[1]': N_0,
    'pq_func_par2[1]': nu, 

    #gamma
    'pq_func_par0[2]': gamma_0,
    'pq_func_par2[2]': gamma_1, 

    #peak mass
    'pq_func_par0[3]': logM_0,
    'pq_func_par2[3]': mu, 
}

In [17]:
# Initialize a fitter object and give it the data to be fit
fitter_smf = ares.inference.FitGalaxyPopulation(**base_pars)

fitter_smf.include.append('smf')

# The data can also be provided more explicitly
#I seem to need this or else the run throws error: Must set data by hand! 
# fitter_lf.redshifts = {‘lf’: [5.9]}
fitter_smf.data = 'tomczak2014',  'mortlock2011', 'moustakas2013', 'marchesini2009_10'

# print(fitter_smf.data)

In [18]:
fitter = ares.inference.ModelFit(**base_pars)
fitter.add_fitter(fitter_smf)

# Establish the object to which we'll pass parameters
from ares.populations.GalaxyHOD import GalaxyHOD
fitter.simulator = GalaxyHOD

In [19]:
# print(fitter.blob_names)
# print(fitter.blob_ivars)
# print(fitter.blob_funcs, fitter.blob_kwargs)


In [21]:
fitter.save_hmf = True  # cache HMF for a speed-up!
fitter.save_psm = True  # cache source SED model (e.g., BPASS, S99)

# Setting this flag to False will make ARES generate new files for each checkpoint.
# 2-D blobs can get large, so this allows us to just download a single
# snapshot or two if we'd like (useful if running on remote machine)
fitter.checkpoint_append = False

fitter.parameters = free_pars
# fitter.is_log = is_log
fitter.prior_set = ps

# In general, the more the merrier (~hundreds)
fitter.nwalkers = 76

# fitter.jitter = [0.1] * len(fitter.parameters)
fitter.jitter = [0.1, 0.1, 0.01, 0.05, 0.1, 0.1, 0.5, 0.05]

fitter.guesses = guesses
# fitter.debug('True')
# fitter.pops

# Fixing position of walker 1 (parameter pq_func_par0[1])
# Moved from 0.0008002595478607023 to 0.013906983915112206
# Fixing position of walker 7 (parameter pq_func_par0[2])
# Moved from 0.7347811099584758 to 0.5824320965473153
# Fixing position of walker 8 (parameter pq_func_par0[2])
# Moved from 0.7901015320201439 to 0.6729590488197127
# Fixing position of walker 38 (parameter pq_func_par0[2])
# Moved from 0.7690715351840084 to 0.6070026204109755
# Fixing position of walker 54 (parameter pq_func_par0[2])
# Moved from 0.7492236395349247 to 0.44269304855143127
# Fixing position of walker 62 (parameter pq_func_par0[2])
# Moved from 0.7078211306531415 to 0.5721739248229668
# Fixing position of walker 63 (parameter pq_func_par0[2])
# Moved from 0.7756947264403903 to 0.46679575348016494
# Fixing position of walker 66 (parameter pq_func_par0[2])
# Moved from 0.8863482624765465 to 0.5966859466129827
# Fixing position of walker 10 (parameter pq_func_par0[3])
# Moved from 12.699422671034442 t

In [ ]:
# Run the thing
fitter.run('test_smfcal', burn=10, steps=70, save_freq=1, clobber=True)

Saved HaloMassFunction instance to limit I/O.
# Starting burn-in: Fri Jun 19 15:15:46 2020
# Wrote test_smfcal.burn.dd0000.facc.pkl: Fri Jun 19 15:21:06 2020
# Wrote test_smfcal.burn.dd0001.facc.pkl: Fri Jun 19 15:23:32 2020
# Wrote test_smfcal.burn.dd0002.facc.pkl: Fri Jun 19 15:25:46 2020
# Wrote test_smfcal.burn.dd0003.facc.pkl: Fri Jun 19 15:27:53 2020
Error, bin(s) out of interpolation bounds
Error, bin(s) out of interpolation bounds
Error, bin(s) out of interpolation bounds
Error, bin(s) out of interpolation bounds
Error, bin(s) out of interpolation bounds
Error, bin(s) out of interpolation bounds
Error, bin(s) out of interpolation bounds
Error, bin(s) out of interpolation bounds
Error, bin(s) out of interpolation bounds
Error, bin(s) out of interpolation bounds
Error, bin(s) out of interpolation bounds
Error, bin(s) out of interpolation bounds
Error, bin(s) out of interpolation bounds
Error, bin(s) out of interpolation bounds
Error, bin(s) out of interpolation bounds
# Wrote tes

In [ ]:
anl = ares.analysis.ModelSet('test_smfcal')

# ax = anl.Scatter(anl.parameters)


In [ ]:
#look at the raw LF samples
# ax = anl.ReconstructedFunction('galaxy_lf', ivar=[6, None], samples='all', color='b', alpha=0.01)

# ax.set_yscale('log')
z = 1.75
# 0.35, 0.875, 1.125, 1.75, 2.25, 2.75

gpop = ares.analysis.GalaxyPopulation()

ax = anl.ReconstructedFunction('galaxy_smf', ivar=[z, None], samples='all', color='b', alpha=0.01)

# Plot any data within dz=0.1 of z=6
gpop.PlotSMF(z, ax=ax, round_z=0.2)

ax.set_ylim(1e-9, 1)
ax.legend()
pl.show()

In [ ]:
params = \
['pq_func_par0[0]',
    'pq_func_par2[0]', 

    #norm
    'pq_func_par0[1]',
    'pq_func_par2[1]', 

    #gamma
    'pq_func_par0[2]',
    'pq_func_par2[2]', 

    #peak mass
    'pq_func_par0[3]',
    'pq_func_par2[3]' 
]


anl.TrianglePlot(pars=params, ivar=[z, None, None, None, None, None, None, None], \
                 label_panels=params) #, #samples='all', color='b', alpha=0.01)


In [ ]:
# WalkerTrajectoriesMultiPlot()

for x in params:
    anl.WalkerTrajectories(x)
    pl.show()